In [22]:
import pandas as pd
from prophet import Prophet

In [13]:
# Lee el archivo como un DataFrame
df  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1000)

# Muestra las primeras filas del DataFrame
print(df.head())
print(df_pid_validos.head())

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
   product_id
0       20001
1       20002
2       20003
3       20004
4       20005


In [15]:
product_ids_validos = df_pid_validos['product_id'].tolist()

In [16]:
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 157.3 MB


In [18]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
data = df[df['product_id'].isin(product_ids_validos)]

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 140.0 MB


In [20]:
# Convertir 'periodo' a formato de fecha
#data['periodo'] = pd.to_datetime(data['periodo'], format='%Y%m')

# Función para agregar los valores de 'tn' por 'product_id' y 'periodo'
def aggregate_data(df):
    return df.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()

# Agrupar los datos
data_aggregated = aggregate_data(data)

In [21]:
data_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22349 entries, 0 to 22348
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  22349 non-null  int64  
 1   periodo     22349 non-null  int64  
 2   tn          22349 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 523.9 KB


In [23]:
# Lista para almacenar los resultados
results = []

# Obtener la lista única de 'product_id'
product_ids = data_aggregated['product_id'].unique()

# Iterar sobre cada 'product_id'
for product_id in product_ids:
    # Filtrar los datos agregados para el 'product_id' actual
    df_product = data_aggregated[data_aggregated['product_id'] == product_id]
    df_product.set_index('periodo',inplace=True)   
    
    # Formatear los datos para Prophet
    #ts = pd.DataFrame({'ds': df_product['periodo'], 'y': df_product['tn']})

    ts = pd.DataFrame({'ds':df.index,'y': df_product['tn']})
    
    # Crear y entrenar el modelo Prophet
    model = Prophet()
    model.fit(ts)
    
    # Crear DataFrame futuro para predicción hasta febrero de 2020
    future = model.make_future_dataframe(periods=2, freq='M')  # Asumiendo que tenemos datos hasta 2019-12
    forecast = model.predict(future)
    
    # Filtrar la predicción para febrero de 2020
    february_forecast = forecast[forecast['ds'] == '2020-02-29']  # Utilizando el último día del mes
    
    # Añadir el 'product_id' al DataFrame de predicciones
    february_forecast['product_id'] = product_id
    
    # Añadir las predicciones a los resultados
    results.append(february_forecast[['ds', 'product_id', 'yhat', 'yhat_lower', 'yhat_upper']])

# Concatenar todos los resultados en un solo DataFrame
final_results = pd.concat(results)

# Guardar el resultado en un archivo CSV
final_results.to_csv('salida_prophet_v1.0.csv', index=False)

print("Predicciones guardadas en 'salida_prophet_v1.0.csv'")

ValueError: array length 2945818 does not match index length 36